In [ ]:
import pandas as pd

def xlsx_to_csv(file_path):
    # Ler o arquivo XLSX usando o pandas
    df = pd.read_excel(file_path)

    # Salvar o DataFrame em um arquivo CSV
    csv_file_path = file_path.replace('.xlsx', '.csv')
    df.to_csv(csv_file_path, index=False)

    print(f'Arquivo CSV salvo em: {csv_file_path}')

# Exemplo de uso
xlsx_file_path = 'Estação_Veneza_2021_-_Ipatinga_-_Usiminas.xlsx'
xlsx_to_csv(xlsx_file_path)

In [ ]:
import os

def preencher_espacos_vazios_com_mediana(file_path):
    # Ler o arquivo CSV usando o pandas
    df = pd.read_csv(file_path)

    # Calcular a mediana das colunas PM10, NO2 e "PM 2,5"
    median_pm10 = df['PM10'].median()
    median_no2 = df['NO2'].median()
    median_pm25 = df['PM 2,5'].median()

    # Preencher espaços vazios nas colunas PM10, NO2 e "PM 2,5" com a mediana
    df['PM10'].fillna(median_pm10, inplace=True)
    df['NO2'].fillna(median_no2, inplace=True)
    df['PM 2,5'].fillna(median_pm25, inplace=True)

    if os.path.exists(file_path):
        os.remove(file_path)
        
    # Salvar o DataFrame atualizado em um novo arquivo CSV
    csv_file_path = file_path.replace('.csv', '_preenchido.csv')
    df.to_csv(csv_file_path, index=False)

    print(f'Arquivo CSV com espaços vazios preenchidos pela mediana salvo em: {csv_file_path}')

# Exemplo de uso
csv_file_path = 'Estação_Paracatu_2021_FR_preenchido_preenchido.csv'
preencher_espacos_vazios_com_mediana(csv_file_path)


In [ ]:
import numpy as np

def preencher_segundos_faltantes(file_path, columns_with_seconds):
    # Ler o arquivo CSV usando o pandas
    df = pd.read_csv(file_path)

    # Verificar colunas com segundos
    for column in columns_with_seconds:
        # Converter a coluna para o tipo datetime
        df[column] = pd.to_datetime(df[column], errors='coerce')

        # Obter os índices dos valores nulos na coluna
        null_indices = df[df[column].isnull()].index

        # Preencher os segundos faltantes com 0
        df[column] = df[column].fillna(pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:00'))

    # Excluir o arquivo anterior, se existir
    if os.path.exists(file_path):
        os.remove(file_path)

    # Salvar o DataFrame atualizado em um novo arquivo CSV
    csv_file_path = file_path.replace('_preenchido.csv', '.csv')
    df.to_csv(csv_file_path, index=False)

    print(f'Arquivo CSV com segundos faltantes preenchidos salvo em: {csv_file_path}')
    
# Exemplo de uso
csv_file_path = 'Estação_Veneza_2021_-_Ipatinga_-_Usiminas_preenchido_preenchido.csv'
preencher_espacos_vazios_com_mediana(csv_file_path)


In [ ]:
def calcular_media_diaria(file_path):
    # Ler o arquivo CSV usando o pandas
    df = pd.read_csv(file_path)

    # Converter a coluna "DATA" para o tipo datetime
    df['DATA'] = pd.to_datetime(df['DATA'])

    # Extrair a data (ignorando a hora, minuto e segundo)
    df['DATA'] = df['DATA'].dt.date

    # Calcular a média diária das colunas PM10, NO2 e "PM 2,5"
    colunas = ['PM10', 'NO2', 'PM 2,5']
    df_media_diaria = df.groupby('DATA')[colunas].mean().round(3).reset_index()

    # Excluir o arquivo anterior, se existir
    if os.path.exists(file_path):
        os.remove(file_path)
        
    # Salvar o DataFrame com a média diária em um novo arquivo CSV
    csv_file_path = file_path.replace('_preenchido_preenchido.csv', '_media_diaria.csv')
    df_media_diaria.to_csv(csv_file_path, index=False)

    print(f'Arquivo CSV com a média diária (limitada a 3 casas decimais) salvo em: {csv_file_path}')
    
csv_file_path = 'Estação_Veneza_2021_-_Ipatinga_-_Usiminas_preenchido_preenchido.csv'
calcular_media_diaria(csv_file_path)


In [6]:
import pandas as pd
import os

def criar_arquivos_agregados():
    # Caminho da pasta de origem e destino (mesmo caminho do código)
    pasta_origem = '.'
    pasta_destino = '.'

    # Lista os arquivos na pasta de origem
    arquivos = os.listdir(pasta_origem)

    # Exclui os diretórios específicos da lista de arquivos
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('.git')]
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('.ipynb_checkpoints')]
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('geo.rar')]
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('medias.ipynb')]
    arquivos = [arquivo for arquivo in arquivos if not arquivo.endswith('Geoespaciais.ipynb')]
    
    # Cria um DataFrame vazio para cada arquivo
    dfs = []
    for arquivo in arquivos:
        arquivo_path = os.path.join(pasta_origem, arquivo)
        df = pd.read_csv(arquivo_path, encoding='latin-1')
        dfs.append(df)

    # Concatena os DataFrames na horizontal
    df_concatenado = pd.concat(dfs, axis=1)

    # Cria os três arquivos CSV com as colunas agregadas
    colunas = ['PM10', 'NO2', 'PM 2,5']
    for coluna in colunas:
        colunas_selecionadas = [f'{coluna}' for i in range(len(arquivos))]
        df_selecionado = df_concatenado[colunas_selecionadas]
        arquivo_saida = os.path.join(pasta_destino, f'{coluna}.csv')
        df_selecionado.to_csv(arquivo_saida, index=False, float_format='%.3f')  # Limita a 3 casas decimais

        print(f'Arquivo {arquivo_saida} criado com sucesso.')

# Exemplo de uso
criar_arquivos_agregados()


Arquivo .\PM10.csv criado com sucesso.
Arquivo .\NO2.csv criado com sucesso.
Arquivo .\PM 2,5.csv criado com sucesso.


In [12]:
import pandas as pd
arquivo_path = 'PM10.csv'
# Carregar o arquivo CSV
df = pd.read_csv(arquivo_path)

# Obter o índice da coluna "PM10.12"
indice_inicio_remocao = df.columns.tolist().index('PM10.12')

# Remover as colunas a partir de "PM10.12"
df = df.iloc[:, :indice_inicio_remocao]
if os.path.exists(arquivo_path):
    os.remove(arquivo_path)

# Salvar o DataFrame modificado em um novo arquivo CSV
df.to_csv(arquivo_path, index=False)

In [17]:
import pandas as pd

arquivo_path = 'PM2,5.csv'
# Carregar o arquivo de origem
df_origem = pd.read_csv('Ipatinga_Estação_Cariru_2021_-_Ipatinga_-_Usiminas_media_diaria.csv')

# Extrair a coluna "DATA"
coluna_data = df_origem['DATA']

# Carregar o arquivo de destino
df_destino = pd.read_csv(arquivo_path)

# Inserir a coluna "DATA" como a primeira coluna
df_destino.insert(0, 'DATA', coluna_data)

if os.path.exists(arquivo_path):
    os.remove(arquivo_path)

# Salvar o DataFrame modificado em um novo arquivo CSV
df_destino.to_csv(arquivo_path, index=False)


In [13]:
import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv('PM2,5.csv')

# Calcular a média das colunas PM2.5, PM2.5.1, PM2.5.2 e criar uma nova coluna
df['PM2.5_media1'] = df[['PM 2.5', 'PM 2.5.1', 'PM 2.5.2']].mean(axis=1)

# Calcular a média das colunas PM2.5.3, PM2.5.4, PM2.5.5, PM2.5.6 e criar uma nova coluna
df['PM2.5_media2'] = df[['PM 2.5.3', 'PM 2.5.4', 'PM 2.5.5', 'PM 2.5.6']].mean(axis=1)

df['PM2.5.7'] = df[['PM 2.5.7']]

# Calcular a média das colunas PM2.5.8, PM2.5.9, PM2.5.10, PM2.5.11 e criar uma nova coluna
df['PM2.5_media3'] = df[['PM 2.5.8', 'PM 2.5.9', 'PM 2.5.10', 'PM 2.5.11']].mean(axis=1)

# Selecionar apenas as colunas desejadas
df_final = df[['DATA', 'PM2.5_media1', 'PM2.5_media2', 'PM2.5.7', 'PM2.5_media3']]

# Salvar o DataFrame modificado em um novo arquivo CSV
df_final.to_csv('PM2.5_editado.csv', index=False)
